In [48]:
!pip install seqeval

  Preparing metadata (setup.py) ... one
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 24.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 37.2 MB/s eta 0:00:0000:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6d28a1cef3d506e698f19a6e9922103fe74a9d786bcf741d357c7a3dd30a1fbd
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#device = 'cpu'

In [ ]:
label_list = ["O","B","I"]
id2label = {
    0: "O",
    1: "B",
    2: "I",
}

label2id = {
    "O": 0,
    "B": 1,
    "I": 2,
}

In [ ]:
dataset = datasets.load_dataset("hodza/BlackBox.Shkola.2014")['train']

In [ ]:
dataset

In [ ]:


lines = [[line, "/Mod/" in fname]  for text, fname in zip(dataset['texts'], dataset['names']) for line in text.split('\n') if len(line) > 10 and not line.isspace()]

is_code = [is_code for l, is_code in lines]
lines = [l for l, is_code in lines]

dataset = datasets.Dataset.from_dict({"texts" : lines, "is_code" : is_code})

In [ ]:
def labelize(sample):
    tkns = sample['texts'].split()
    if sample['is_code']:
        labels = [label2id['I'] for x in tkns]
        labels[0] = label2id['B']
    else:
        labels = [label2id['O'] for x in tkns]
    return {"tokens" :tkns, "tags" : labels}


dataset = dataset
dataset = dataset.map(labelize)
dataset = dataset.train_test_split()

In [ ]:
print(dataset['train'][140]['texts'], dataset['train'][140]['tags'])

In [ ]:
model_name = "distilbert/distilbert-base-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)


In [ ]:
model = transformers.AutoModelForTokenClassification.from_pretrained(model_name, device_map=device, num_labels=3, id2label=id2label, label2id=label2id)

In [ ]:
data_collator = transformers.DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np
#labels = [label_list[i] for i in dataset[f"tags"]]
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [20]:
training_args = transformers.TrainingArguments(
    output_dir="./results/code_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps = 1,
    load_best_model_at_end=True,
    push_to_hub=False,
 
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.089645,0.895833,0.945055,0.919786,0.976318
2,No log,0.102540,0.878788,0.956044,0.915789,0.976737


TrainOutput(global_step=100, training_loss=0.06323957920074463, metrics={'train_runtime': 48.3513, 'train_samples_per_second': 32.678, 'train_steps_per_second': 2.068, 'total_flos': 206435630592000.0, 'train_loss': 0.06323957920074463, 'epoch': 2.0})

In [47]:
test = """
MODULE  AAA;
IMPORT StdLog;
PROCEDURE AAA;
BEGIN
END AAA;
END AAA.
"""

In [59]:
words = test.split()

inputs = tokenizer(words, return_tensors="pt", truncation=True, is_split_into_words=True).to(device)
word_ids = inputs.word_ids(batch_index=0)

with torch.no_grad():
    logits = model(**inputs).logits
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

tags = []
prev_word_id = None
for wid, tag in zip(word_ids, predicted_token_class):
    if wid != prev_word_id:
        if prev_word_id is not None:
            print(prev_word_id, words[prev_word_id], tags)
        prev_word_id = wid
        tags = []
    else:
        tags.append(tag)
        


0 MODULE ['O', 'O', 'O']
1 AAA; ['O']
2 IMPORT ['O', 'O', 'O']
3 StdLog; ['O', 'O', 'O', 'O']
4 PROCEDURE ['O', 'O', 'O', 'O']
5 AAA; ['O']
6 BEGIN ['O', 'O']
7 END ['O']
8 AAA; ['O']
9 END ['O']
10 AAA. ['O']
